In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn import preprocessing
import warnings
import time
warnings.filterwarnings("ignore")
import lightgbm as lgb
from bayes_opt import BayesianOptimization
from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.model_selection import train_test_split

# Importing the dataset
dataset  = pd.read_csv('C:\Fires\FullDatasetNoArson.csv')

# Model Vars
objective = 'binary'
metric = [ 'binary_logloss', 'auc']
sub_feature = 0.5721
num_leaves = 1080
min_data = 200
max_depth = 14
max_bin = 300


In [6]:
x = dataset.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler.fit(x)
x_scaled = min_max_scaler.transform(x)
dataset = pd.DataFrame(x_scaled)
vals = dataset.values
X = dataset.iloc[:, 0:8].values
y = dataset.iloc[:, 8].values

In [7]:
def bayesion_opt_lgbm(X, y, init_iter=3, n_iters=7, random_state=11, seed = 101, num_iterations = 100):
    dtrain = lgb.Dataset(data=X, label=y)
    
# Objective Function
    def hyp_lgbm(num_leaves, sub_fraction, max_depth, max_bin, min_data):
        
        params = {'objective': objective,'boosting_type':'gbdt','num_iterations': num_iterations,
                    'learning_rate':0.05, 'early_stopping_round':50,'metric': metric,
                    'metric':'auc'} 
            # Default parameters
        params["num_leaves"] = int(round(num_leaves))
        params['sub_fraction'] = max(min(sub_fraction, 1), 0)
        params['max_bin'] = int(round(max_bin))
        params['max_depth'] = int(round(max_depth))
        params['min_data'] = int(round(min_data))
        cv_results = lgb.cv(params,dtrain, nfold=5, seed=seed,categorical_feature=[], stratified=False,verbose_eval =None)
        # print(cv_results)
        return np.max(cv_results['auc-mean'])
  
    # Domain space-- Range of hyperparameters 
    pds = {'num_leaves': (720, 2048),
            'sub_fraction': (0.55, 0.56),
            'max_depth': (15, 19),
           'max_bin': (1000,2000),
           'min_data' : (400,600),
            }

  # Surrogate model
    optimizer = BayesianOptimization(hyp_lgbm, pds, random_state=random_state)
                                    
  # Optimize
    optimizer.maximize(init_points=init_iter, n_iter=n_iters)

In [8]:
bayesion_opt_lgbm(X, y, init_iter=10, n_iters=40, random_state=77, seed = 101, num_iterations = 2000)

|   iter    |  target   |  max_bin  | max_depth | min_data  | num_le... | sub_fr... |
-------------------------------------------------------------------------------------
|  1        |  0.8211   |  1.919e+0 |  17.57    |  550.7    |  905.0    |  0.5509   |
|  2        |  0.821    |  1.788e+0 |  16.3     |  508.2    |  1.039e+0 |  0.5555   |
|  3        |  0.8206   |  1.401e+0 |  17.86    |  567.3    |  1.502e+0 |  0.553    |
|  4        |  0.8231   |  1.281e+0 |  17.82    |  484.5    |  796.1    |  0.5575   |
|  5        |  0.8239   |  1.452e+0 |  15.7     |  409.9    |  1.108e+0 |  0.5507   |
|  6        |  0.8206   |  1.751e+0 |  15.26    |  486.4    |  1.204e+0 |  0.5515   |
|  7        |  0.8248   |  1.547e+0 |  16.77    |  407.2    |  1.813e+0 |  0.5527   |
|  8        |  0.8251   |  1.169e+0 |  17.58    |  421.5    |  1.189e+0 |  0.5539   |
|  9        |  0.8218   |  1.446e+0 |  18.91    |  545.9    |  1.135e+0 |  0.5589   |
|  10       |  0.8224   |  1.783e+0 |  16.05    |  461

KeyboardInterrupt: 